In [1]:
import numpy as np
from data_iterators import *

In [2]:
# Use RunIteratorSettins to create a type of iterator you want

settings = RunIteratorSettings(
      zbynek_mode = False 
    , remove_duplicits_from_population = True                               
    , remove_duplicits_from_archive = True                                  
    , remove_already_known_points_from_tss_selection_process = False 
    , remove_already_known_points_from_evaluation_process = True            
    , tss = 0 # avaiable = {0,2}   
    , use_distance_weight = True                                            
)



In [3]:
# create an iterator - optionaly pass some filters

if False:
    iterator = RunIterator(settings)
elif False:
    filters = lambda x: '2D' in x.name 
    iterator = RunIterator(settings, filters=filters)
else:
    filters = [
        lambda x: '2D' in x.name ,
        lambda x: 2 < len(x.gen_split) < 5
    ]
    iterator = RunIterator(settings, filters=filters)

In [4]:
# use the iterator: 
for run in iterator:
    print(f'Found {run}')



Found exp_doubleEC_28_log_nonadapt_results_5_2D_38_0.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_39_0.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_39_2.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_39_4.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_40_0.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_40_1.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_40_4.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_41_0.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_41_1.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_41_2.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_41_3.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_43_1.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_43_3.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_43_4.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_44_1.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_44_3.npz
Found exp_doubleEC_28_log_nonadapt_results_5_2D_45_2.npz
Found exp_doubleEC_28_log_nonad

In [5]:
# this prints all attributes of a run
for attr in dir(run):
    if attr.startswith('_'):
        continue
    a = getattr(run,attr)
    if isinstance(a, np.ndarray):
        print(f'{attr}: {a.shape}')
    else:
        print(f'{attr}: {a}')

dimensions: ()
evals: 9
exp_id: ()
function_id: ()
fvalues: (37,)
fvalues_orig: (37,)
gen_split: (3,)
iruns: (3,)
name: exp_doubleEC_28_log_nonadapt_results_5_2D_45_4.npz
orig_evaled: (37,)
path: ../../npz-data/exp_doubleEC_28_log_nonadapt_results_5_2D_45_4.npz
points: (37, 2)
restarts: ()
settings: <data_iterators.RunIteratorSettings object at 0x7ff4ad2fdb00>
surrogate_data_bds: (3, 2, 2)
surrogate_data_means: (2, 3)
surrogate_data_sigmas: (3,)
surrogate_param_range: ()
surrogate_param_set_size_max: ()
surrogate_param_type: ()


In [6]:
# each run has a bunch of states on which a regression algorithm should be trained...
for run in iterator:
    print(f"RUN - {run} " + "-" * 30)
    for state in run:
        print(f"STATE  ")

RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_38_0.npz ------------------------------
STATE  
RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_39_0.npz ------------------------------
STATE  
RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_39_2.npz ------------------------------
STATE  
STATE  
RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_39_4.npz ------------------------------
STATE  
RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_40_0.npz ------------------------------
STATE  
RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_40_1.npz ------------------------------
STATE  
RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_40_4.npz ------------------------------
STATE  
RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_41_0.npz ------------------------------
STATE  
RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_41_1.npz ------------------------------
STATE  
RUN - exp_doubleEC_28_log_nonadapt_results_5_2D_41_2.npz ------------------------------
STATE  
RUN - exp_doubleEC_28_log_nonada

In [7]:
# states have attributes...
print(f"------------ state {state._gen} ---------------")
for attr in dir(state):
    if isinstance(getattr(state, attr), np.ndarray):
        print(f"state.{attr} - {getattr(state, attr).shape}")

------------ state 1 ---------------
state.distances - (6,)
state.x_eval - (25, 2)
state.x_fit - (6, 2)
state.y_eval - (25,)
state.y_fit - (6,)


In [8]:
# so the final pattern should be 

settings = RunIteratorSettings(
      zbynek_mode = False 
    , remove_duplicits_from_population = True                               
    , remove_duplicits_from_archive = True                                  
    , remove_already_known_points_from_tss_selection_process = False 
    , remove_already_known_points_from_evaluation_process = True            
    , tss = 0 # avaiable = {0,2}   
    , use_distance_weight = True                                            
)

for run in RunIterator(settings):
    break # remove this line :)
    for state in run:
        # CREATE REGRESSION MODEL
        # EVALUATE
        
        state.pass_result('name', 'result') # NotImplemented ...